In [11]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
# from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRFClassifier

In [2]:
arff_train,arff_train_meta=arff.loadarff('C:\\Users\Administrator\Downloads\data\\nslkdd\KDDTrain+.arff')
arff_test,_=arff.loadarff('C:\\Users\Administrator\Downloads\data\\nslkdd\KDDTest+.arff')

nslkdd_train_data=pd.DataFrame(data=arff_train,columns=arff_train.dtype.names)
test_data=pd.DataFrame(data=arff_test,columns=arff_test.dtype.names)
nslkdd_train_y=nslkdd_train_data['class']
nslkdd_train_x=nslkdd_train_data.drop(['class'],axis=1)
y_test=test_data['class']
x_test=test_data.drop(['class'],axis=1)

In [3]:
x_train,x_valid,y_train,y_valid=train_test_split(nslkdd_train_x,nslkdd_train_y,train_size=0.7,random_state=0)

In [4]:
col=(nslkdd_train_x.dtypes==object)
object_col=list(col[col].index)
col=(nslkdd_train_x.dtypes==float)
float_col=list(col[col].index)

In [5]:
ohe=prep.OneHotEncoder(handle_unknown='ignore',sparse=False)
ohe_col_train=pd.DataFrame(ohe.fit_transform(x_train[object_col]))
ohe_col_valid=pd.DataFrame(ohe.transform(x_valid[object_col]))
ohe_col_test=pd.DataFrame(ohe.transform(x_test[object_col]))

ohe_col_train.index=x_train.index
ohe_col_valid.index=x_valid.index
ohe_col_test.index=x_test.index

ohe_train_x=pd.concat([x_train[float_col],ohe_col_train],axis=1)
ohe_valid_x=pd.concat([x_valid[float_col],ohe_col_valid],axis=1)
x_test=pd.concat([x_test[float_col],ohe_col_test],axis=1)

In [6]:
ylab=prep.LabelEncoder()

y_train=ylab.fit_transform(y_train)
y_valid=ylab.transform(y_valid)
y_test=ylab.transform(y_test)

In [16]:
model=XGBRFClassifier(n_estimators=100,learning_rate=0.04)
model.fit(ohe_train_x,y_train)

predictions=model.predict(ohe_valid_x)
print("valid mean abs error: "+str(mean_absolute_error(predictions,y_valid)))
print(pd.Series(predictions==y_valid).value_counts())
print()

predictions=model.predict(x_test)
print("test mean abs error: "+str(mean_absolute_error(predictions,y_test)))
print(pd.Series(predictions==y_test).value_counts())

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:07:21] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
valid mean abs error: 0.0070385266723116
True     37526
False      266
dtype: int64

test mean abs error: 0.20830376153300212
True     17848
False     4696
dtype: int64
